# This is the script for the linear regression model
## The imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import FinalBorealis as dg
import datetime

I decided to create the module as a class so I could give it an input size and output size, and for future reusability

In [2]:
class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)
    def forward(self, x):
        return self.linear(x)

This is how the training of the model is done. For this function I used inspiration from the example model given by Borealis, this function receives:
    - Model: In this case a linear regression model.
    - Opt: Optimization function.
    - loss_fn: Loss Function.
    - train_ld: the data.
    - n_epochs (default = 3): Epochs of training.
    - save_to (default = './trainedModel.pth'): The file generated for the model

In [88]:
def train(model, opt, loss_fn, train_ld, n_epoch=3, save_to='./trainedModel.pth'):
    ##Iterate through a number of epochs
    for epoch in range(n_epoch):
        ##Training with batches of data
        running_loss = 0.0
        num_iters = len(train_ld)
        for i, data in enumerate(train_ld,0):
            print(type)
            inputs, labels = data[:,:16], data[:,16:]
            #First step: Generate predictions
           
            pred = model(inputs)
           
            #2nd step: Calculate loss
            loss = loss_fn(pred, labels)
            loss.backward()
            opt.step()

            opt.zero_grad()
            running_loss += loss.item()
            if i % 1014 == 1013:
                print(f"Loss at epoch [{epoch}/{n_epoch}], iteration [{i + 1}/{num_iters}]:{running_loss /2000}")
                running_loss = 0.0
    print(f"Training done after {n_epoch} epochs!")

    torch.save(model.state_dict(), save_to)
    print(f"Trained model is saved to {save_to}.")


Converting the pandas dataframe to a torch tensor

In [52]:
def pdToTensor(dataframe, file=True):
    if not file:
        dataRefined = dg.filteredByHour(data)
        ingredients = { 'tuna':'TunaWeight', 'meatball':'MeatballWeight', 'chicken':'ChickenWeight',
                'steak':'SteakWeight', 'chickenTeriyaki':'ChcknTkiWeight', 'cheese':'Cheese', 'tomato':'Tomato', 'olives':'Olives',
                'avocado':'Avocado'}
        finalDF = pd.DataFrame(columns=['Weather','Day','Hour','TypeOfFood','output'] )
        for row in dataRefined.iterrows():
            sample = {'Weather': row[1].Weather,'Day':row[0].dayofyear,'Hour':row[0].hour}
            for ing in ingredients:
                if row[1][ingredients[ing]] != 0:
                    sample['TypeOfFood'] = ing
                    sample['output'] = row[1][ingredients[ing]]
                    finalDF = finalDF.append(sample, ignore_index=True)
        finalDF = pd.concat([pd.get_dummies(finalDF['Weather'], prefix='', prefix_sep=''), finalDF], axis=1)
        finalDF = pd.concat([pd.get_dummies(finalDF['TypeOfFood'], prefix='', prefix_sep=''), finalDF], axis=1)
        del finalDF['Weather']
        del finalDF['TypeOfFood']
        finalDF.to_json(f'/data{datetime.now}.json')
        return torch.from_numpy(finalDF.values)
    
    newTensor = torch.from_numpy(dataframe.values)
    return newTensor

In [94]:
def main():
    ## TO DO: implement a tensorBoard to visualize the loss
    writer = SummaryWriter()
    
    data = pdToTensor(pd.read_json('data.json'))
    ## Converting the dataframe to a tensor 
    data = data.float()
    #inputs (date/hour, type of ingredient)
    testSize = int(0.1 * len(data))
    trainSize = len(data)-testSize

    trainSet, testSet = torch.utils.data.random_split(data, [trainSize, testSize])

    trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=4, num_workers=0)
    testLoader = torch.utils.data.DataLoader(testSet, batch_size=4, num_workers=0)
    
    print(f"Training set consist of {len(trainSet)}, and the test set consists of {len(testSet)}")


    model = linearRegression(16,1)
    loss_fn = nn.MSELoss()
    opt = torch.optim.SGD(model.parameters(), lr=1e-5)
    #print(model.parameters)
    train(model, opt, loss_fn, trainLoader, n_epoch=5)

In [95]:
if __name__ == '__main__':
    main()

Training set consist of 27262, and the test set consists of 3029
Loss at epoch [0/5], iteration [1014/6816]:15473.319845672608
Loss at epoch [0/5], iteration [2028/6816]:15727.631386108398
Loss at epoch [0/5], iteration [3042/6816]:14651.931653305053
Loss at epoch [0/5], iteration [4056/6816]:15741.96657006073
Loss at epoch [0/5], iteration [5070/6816]:15094.247301742555
Loss at epoch [0/5], iteration [6084/6816]:15346.470744476319
Loss at epoch [1/5], iteration [1014/6816]:14185.953639221192
Loss at epoch [1/5], iteration [2028/6816]:15497.017599975587
Loss at epoch [1/5], iteration [3042/6816]:14454.089000610351
Loss at epoch [1/5], iteration [4056/6816]:15553.20206641388
Loss at epoch [1/5], iteration [5070/6816]:14911.949539337158
Loss at epoch [1/5], iteration [6084/6816]:15151.304972015381
Loss at epoch [2/5], iteration [1014/6816]:14007.172763496399
Loss at epoch [2/5], iteration [2028/6816]:15305.734486221314
Loss at epoch [2/5], iteration [3042/6816]:14267.099618240356
Loss at